In [14]:
from minio.error import S3Error
from pyspark.sql import SparkSession
from delta import *
from pyspark.sql import functions as f

import os
from apache_atlas.utils.Constants import EndRelations, TypeNames
from apache_atlas.utils.API import API, HTTPMethod
from apache_atlas.client.ApacheAtlas import ApacheAtlasClient
import re
import json
import logging

In [15]:
atlas_section = ApacheAtlasClient(
    "http://10.100.100.61:21000",
    "admin",
    "admin"
)

In [16]:
acess_key = "WqmtspUHrxQ4frTx6MS5"
secret_key = "yYmK73ZpZncg1YSRYm6MXLdPo4S9jXRYm8PIq6pR"

spark = SparkSession.builder.master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,"
                                    "org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.hadoop.fs.s3a.access.key", acess_key) \
    .config("spark.hadoop.fs.s3a.secret.key", secret_key) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://10.100.100.61:9000") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.io.native.lib", "false") \
    .getOrCreate()

In [ ]:
response = atlas_section.search.search_by_attribute({
    'typeName': f'{TypeNames.TABLE_FILE}',
    'attrName': 'state',
    'attrValuePrefix': "GO",
    'limit': 99999,
    'offset': 0
})


http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table_file&attrName=state&attrValuePrefix=GO&limit=99999&offset=0
{
  "queryType": "BASIC",
  "searchParameters": {
    "typeName": "dt_table_file",
    "excludeDeletedEntities": false,
    "includeClassificationAttributes": false,
    "includeSubTypes": true,
    "includeSubClassifications": true,
    "excludeHeaderAttributes": false,
    "limit": 100,
    "offset": 0,
    "entityFilters": {
      "attributeName": "state",
      "operator": "startsWith",
      "attributeValue": "GO"
    }
  },
  "entities": [
    {
      "typeName": "dt_table_file",
      "attributes": {
        "owner": "Lucas Ferreira Maia",
        "qualifiedName": "dt_table_file.DataSUS@DOGO2004.dbc",
        "name": "DOGO2004.dbc",
        "description": "Arquivo de Declarac\u00e3o de \u00d3bito de Goi\u00e1s 2004",
        "location": "bronze/SIM/DOGO2004.dbc",
        "state": "GO"
      },
      "guid": "59db1fe9-d9a4-4b9f-bf77-f9772a5e3f8f",

In [30]:
entities = list(filter(lambda x: not x['attributes']['name'].startswith('DO'), response['entities']))
guids = [i['guid'] for i in entities]

entities_full = atlas_section.entity.get_entities_by_guid(guids)

http://10.100.100.61:21000/api/atlas/v2/entity/bulk?guid=9004d60e-c8b1-4785-a5a1-ee0a56a33077&guid=7e2dbf6d-f186-48ec-adc0-2690106370b1&guid=9b894c56-32a5-4d1a-9420-aea94e3e6c41&guid=61c6657c-d4ae-435c-8b44-b220d546cf85&guid=3bc97610-76fd-458d-9af2-9fb58ee66e7c&guid=c1fb22b3-70c3-4e58-8b72-e0ec7d13e4bc&guid=049c2846-5a9f-4579-a06b-715b303cc462&guid=22778c52-492f-42c9-be42-e33f4817eb41&guid=2099d2f9-1e68-4861-81cd-d7be82a5e101&guid=df2db8c3-2da7-475f-8dbe-44c1bc222bc2&guid=0e0ab321-68eb-4acc-a841-0077d1b37f1c&guid=b7ba9226-2ed2-4fb2-8d9f-e8ab4227582a&guid=244f4f57-5507-47c7-a4f2-42f3b6863d0d&guid=bc82953a-836f-450d-abc7-35a70b5db41f&guid=545a50f2-f31a-4ebc-b4f5-626170b54327&guid=69ee4243-c8ed-452a-a127-ae9464acb586&guid=9aae0b8a-c10d-41ce-b168-5258c96a10e6&guid=6ce0f535-50e6-4ac2-a20f-d47eeb5f1095&guid=640a0c49-e602-4a92-8774-9094cfd04c94&guid=19474d55-0012-4d8a-8143-55538d769d03&guid=fbbb6eed-1573-48ed-85f9-7b10aca1617d&guid=9e483da4-6e9a-4c65-9cec-089035442b39&guid=2318f3f6-bf47-4980-

In [ ]:
def extract_file_info(filename: str):
    filename = filename.strip().split('_')[0].upper()
    filename = filename if filename[-1] not in ['A', 'B', 'C'] else filename[:-1]

    match = re.match(r"(^.{2,3})(\w{2})(\d{2})(\d{2}).PARQUET", filename)

    if match:
        sigla = match.group(1)
        state = match.group(2)
        year =  match.group(3)
        month = match.group(4)
        return sigla, state, year, month
    return None, None, None, None

def mapped(entity):
    dados = entity['attributes']
    new_name = dados['name'].split('.')[0] + '.parquet'
    ano = dados['year']
    sigla, state, year, month = extract_file_info(entity['attributes']['name'])

    entity['attributes'] = {
        **entity['attributes'],
        **{
             'name': new_name,
             'location': f'/bronze/sia-parquet/{sigla}/{new_name}',
             'qualifiedName': f'dt_table_file.{sigla}.DataSUS@{new_name}',
             'year': ano if ano > 2000 else int(ano) + 2000
        }     
     }

    return entity 

data = list(map(mapped, entities_full['entities']))

atlas_section.entity.create_multiple_entities(data)

[
  {
    "typeName": "dt_table_file",
    "attributes": {
      "owner": null,
      "extension": ".parquet",
      "replicatedTo": null,
      "userDescription": null,
      "replicatedFrom": null,
      "year": 2018,
      "qualifiedName": "dt_table_file.SAD.DataSUS@SADGO1801.parquet",
      "displayName": null,
      "description": "Arquivo SIA de SADGO1801.PARQUET",
      "total_lines": 28,
      "file_size": "0.03 MB",
      "name": "SADGO1801.parquet",
      "location": "/bronze/sia-parquet/SAD/SADGO1801.parquet",
      "state": "GO",
      "quallifiedName": "dt_table_file.SAD.DataSUS@SADGO1801.parquet"
    },
    "guid": "9004d60e-c8b1-4785-a5a1-ee0a56a33077",
    "isIncomplete": false,
    "status": "ACTIVE",
    "createdBy": "admin",
    "updatedBy": "admin",
    "createTime": 1732103325757,
    "updateTime": 1732103325757,
    "version": 0,
    "relationshipAttributes": {
      "inputToProcesses": [],
      "pipeline": null,
      "schema": [],
      "columns_file_table": {


# Codigos

In [19]:
log_file = "data/processamento.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%H:%M:%S",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
    
def get_parquet_dir_size(dir_path: str) -> str:
    total_size = 0
    for dirpath, _, filenames in os.walk(dir_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)

    # Converter para MB e formatar como string
    size_in_mb = total_size / (1024 ** 2)
    return f"{size_in_mb:.2f} MB"

def extract_file_info(filename: str):
    filename = filename.strip().split('_')[0].upper()
    filename = filename if filename[-1] not in ['A', 'B', 'C'] else filename[:-1]

    match = re.match(r"(^.{2,3})(\w{2})(\d{2})(\d{2}).PARQUET", filename)

    if match:
        sigla = match.group(1)
        state = match.group(2)
        year =  match.group(3)
        month = match.group(4)
        return sigla, state, year, month
    return None, None, None, None

In [22]:
pasta = 'C:/Users/Arlete/Desktop/Dados'

for parquet_file in os.listdir(pasta):
    parquet_file = parquet_file.upper()
    sigla, state, year, month = extract_file_info(parquet_file)

    if sigla is None:
        raise Exception(f"Erro em {parquet_file}")

    colunas = sigla + year + month

    path_leitura = os.path.join(pasta, parquet_file)
    path_remoto = f"s3a://bronze/sia-parquet/{sigla}/{sigla + state + year + month}.parquet"

    print(f"Arquivo {path_remoto} sendo gerado...")

    try:
        df = spark.read.parquet(path_leitura)
        lines = df.count()

        logging.info(f"Arquivo {parquet_file} - {lines}")

        df.write.format("parquet").save(path_remoto)

        logging.info(f"Arquivo {parquet_file} salvo com sucesso [{path_remoto}]")

        ## Atlas
        file_info = {
            "name": parquet_file,
            "description": f"Arquivo SIA de {parquet_file}",
            "extension": ".parquet",
            "file_size": f"{get_parquet_dir_size(path_leitura)}",
            "location": path_remoto.replace("s3a:/", ""),
            "state": state,
            "total_lines": lines,
            "year": year,
            "month": month,
        }

        atlas_section.entity.create_entity_file_table(file_info, sigla, colunas)

        logging.info(f" [Atlas] Arquivo {parquet_file} salvo com sucesso")        
    except Exception as err:
        logging.info(f"fError: [{parquet_file}] - {err}")

Arquivo s3a://bronze/sia-parquet/ACF/ACFGO2401.parquet sendo gerado...
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table_file&attrName=name&attrValuePrefix=ACFGO2401.PARQUET&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table&attrName=acronymus&attrValuePrefix=ACF&offset=0
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_anual_table&attrName=name&attrValuePrefix=ACF2401&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/entity/
Arquivo s3a://bronze/sia-parquet/ACF/ACFGO2402.parquet sendo gerado...
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table_file&attrName=name&attrValuePrefix=ACFGO2402.PARQUET&limit=1&offset=0
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_table&attrName=acronymus&attrValuePrefix=ACF&offset=0
http://10.100.100.61:21000/api/atlas/v2/search/attribute?typeName=dt_anual_table&attrName=name&attrValuePrefix=ACF2402&limit=1&offset=0
ht

# Resultados

In [17]:
remote_path = "s3a://silver/nordeste/DORN2018.parquet"

df = spark.read.format("parquet").load(remote_path)

# df = df.withColumn("timestamp", f.to_timestamp("Data do Óbito", "ddMMyyyy"))
# df = df.orderBy("timestamp")  

print(f"Total de Registros: {df.count()} - ({len(df.columns)}) | {list(df.columns)}")

df.show(truncate=False)


Total de Registros: 122888 - (88) | ['ORIGEM', 'Tipo de Óbito', 'Data do Óbito', 'Hora do Óbito', 'Naturalidade', 'CODMUNNATU', 'Data de Nascimento', 'Idade', 'Sexo', 'Raça/Cor', 'Estado Civil', 'Escolaridade', 'ESC2010', 'SERIESCFAL', 'OCUP', 'CODMUNRES', 'Local da Ocorrência', 'Hospital', 'ESTABDESCR', 'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE', 'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'Assintência Médica', 'EXAME', 'CIRURGIA', 'Necrópsia', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'Causa Básica', 'CB_PRE', 'CRM', 'Cidade do Paciente', 'Data do Atestado', 'CIRCOBITO', 'ACIDTRAB', 'FONTE', 'NUMEROLOTE', 'TPPOS', 'DTINVESTIG', 'CAUSABAS_O', 'Data do Cadastro', 'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB', 'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'CAUSAMAT', 'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA'

In [7]:
df_6 = df.dropDuplicates()

print(f"Total de Registros: {df_6.count()} - ({len(df_6.columns)}) | {list(df_6.columns)}")

df_6.show(truncate=False)

Total de Registros: 122888 - (89) | ['ORIGEM', 'Tipo de Óbito', 'Data do Óbito', 'Hora do Óbito', 'Naturalidade', 'CODMUNNATU', 'Data de Nascimento', 'Idade', 'Sexo', 'Raça/Cor', 'Estado Civil', 'Escolaridade', 'ESC2010', 'SERIESCFAL', 'OCUP', 'CODMUNRES', 'Local da Ocorrência', 'Hospital', 'ESTABDESCR', 'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE', 'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'Assintência Médica', 'EXAME', 'CIRURGIA', 'Necrópsia', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'Causa Básica', 'CB_PRE', 'CRM', 'Cidade do Paciente', 'Data do Atestado', 'CIRCOBITO', 'ACIDTRAB', 'FONTE', 'NUMEROLOTE', 'TPPOS', 'DTINVESTIG', 'CAUSABAS_O', 'Data do Cadastro', 'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB', 'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'CAUSAMAT', 'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA'

In [8]:
remote_path_2 = "s3a://silver/sim-parquet/DOPE2018.parquet"

df_2 = spark.read.format("parquet").load(remote_path_2)

# df_2 = df_2.withColumn("timestamp", f.to_timestamp("Data do Óbito", "ddMMyyyy"))
# df_2 = df_2.orderBy("timestamp") 

print(f"Total de Registros: {df_2.count()} - ({len(df_2.columns)}) | {list(df_2.columns)}")

df_2.show(truncate=False)

Total de Registros: 62011 - (59) | ['Tipo de Óbito', 'Data do Óbito', 'Hora do Óbito', 'Naturalidade', 'Data de Nascimento', 'Idade', 'Sexo', 'Raça/Cor', 'Estado Civil', 'Escolaridade', 'Local da Ocorrência', 'Hospital', 'ESTABDESCR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE', 'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'Assintência Médica', 'EXAME', 'CIRURGIA', 'Necrópsia', 'Causa Básica', 'CB_PRE', 'CRM', 'Cidade do Paciente', 'Data do Atestado', 'CIRCOBITO', 'ACIDTRAB', 'FONTE', 'DTINVESTIG', 'Data do Cadastro', 'FONTEINV', 'CAUSAMAT', 'NUDIASOBCO', 'NUDIASOBIN', 'DTCADINV', 'TPOBITOCOR', 'DTCONINV', 'FONTES', 'TPRESGINFO', 'TPNIVELINV', 'NUDIASINF', 'DTCADINF', 'MORTEPARTO', 'DTCONCASO', 'FONTESINF', 'ALTCAUSA']
+-------------+-------------+-------------+------------+------------------+-------+------+--------+----------------------+------------+-------------------+-------

In [18]:
remote_path_3 = "s3a://bronze/sim-parquet/DOSP2021.parquet"

df_3 = spark.read.format("parquet").load(remote_path_3)

# df_3 = df_3.withColumn("timestamp", f.to_timestamp("DTOBITO", "ddMMyyyy"))
# df_3 = df_3.orderBy("timestamp") 

print(f"Total de Registros: {df_3.count()} - ({len(df_3.columns)}) | {list(df_3.columns)}")

df_3.show(truncate=False)


Total de Registros: 431616 - (87) | ['ORIGEM', 'TIPOBITO', 'DTOBITO', 'HORAOBITO', 'NATURAL', 'CODMUNNATU', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010', 'SERIESCFAL', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODESTAB', 'ESTABDESCR', 'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE', 'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC', 'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CB_PRE', 'COMUNSVOIM', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE', 'NUMEROLOTE', 'TPPOS', 'DTINVESTIG', 'CAUSABAS_O', 'DTCADASTRO', 'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB', 'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'CAUSAMAT', 'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA', 'NUDIASOBCO', 'NUDIASOBIN', 'DTCADINV', 'TPOBITOCOR', 'DTCONINV', 'FONTES', 'TPRESGINFO', 'TPNIVEL